# Enhancer - BC assignment for 500 bp library

In [1]:
cd /staging/leuven/stg_00002/lcb/lcb_projects/CSE/500bp_ATAC_peak_library/enhancer_assignment

In [2]:
module load FastQC/0.11.8-Java-1.8.0_162
module load cutadapt/1.18-foss-2018a-Python-3.7.0
module load SeqKit/0.10.2
module load fastp/0.20.0-foss-2018a


The following have been reloaded with a version change:
  1) Python/3.7.4-foss-2018a => Python/3.7.0-foss-2018a
  2) SQLite/3.29.0-GCCcore-6.4.0 => SQLite/3.21.0-GCCcore-6.4.0
  3) Tcl/8.6.9-GCCcore-6.4.0 => Tcl/8.6.8-GCCcore-6.4.0
  4) XZ/5.2.4-GCCcore-6.4.0 => XZ/5.2.3-GCCcore-6.4.0
  5) bzip2/1.0.8-GCCcore-6.4.0 => bzip2/1.0.6-GCCcore-6.4.0
  6) libreadline/8.0-GCCcore-6.4.0 => libreadline/7.0-GCCcore-6.4.0
  7) ncurses/6.1-GCCcore-6.4.0 => ncurses/6.0-GCCcore-6.4.0



In [6]:
path_raw="/staging/leuven/stg_00002/lcb/ngs_runs/NovaSeq6000_20191213/Demultiplexed/"

## 01.fastqc

In [ ]:
fastqc ${path_raw}CSE__10dc17__CheqSeq_500_bp_LC_S49_L00*_R*_001.fastq.gz -o 01.fastqc

## 02.clean_reads

In [ ]:
# Combine R1 and extract free BC with cutadapt
cat ${path_raw}CSE__10dc17__CheqSeq_500_bp_LC_S49_L00*_R1_001.fastq.gz > 02.clean_reads/CSE__10dc17__CheqSeq_500_bp_LC_R1.fastq.gz
cutadapt -g CGCACTGGTCACGGAG...AGGCTAGGTGGAGGCTCAGTGagctC -m 8 -M 8 -j 10 --discard-untrimmed \
         -o 02.clean_reads/CSE__10dc17__CheqSeq_500_bp_LC_R1_BC.fastq.gz 02.clean_reads/CSE__10dc17__CheqSeq_500_bp_LC_R1.fastq.gz
rm 02.clean_reads/CSE__10dc17__CheqSeq_500_bp_LC_R1.fastq.gz

In [7]:
# Combine R2 and extract plasmid BC with cutadapt then do rev comp
cat ${path_raw}CSE__10dc17__CheqSeq_500_bp_LC_S49_L00*_R2_001.fastq.gz > 02.clean_reads/CSE__10dc17__CheqSeq_500_bp_LC_R2.fastq.gz
cutadapt -g AATTAATTCGGGCCCCGGTCC...GATCGGCGCGCCTGCTCG -m 17 -M 17 -j 10 --discard-untrimmed \
         02.clean_reads/CSE__10dc17__CheqSeq_500_bp_LC_R2.fastq.gz |\
seqkit seq -r -p -o 02.clean_reads/CSE__10dc17__CheqSeq_500_bp_LC_R2_BC.fastq.gz
rm 02.clean_reads/CSE__10dc17__CheqSeq_500_bp_LC_R2.fastq.gz

This is cutadapt 1.18 with Python 3.7.0
Command line parameters: -g AATTAATTCGGGCCCCGGTCC...GATCGGCGCGCCTGCTCG -m 17 -M 17 -j 10 --discard-untrimmed 02.clean_reads/CSE__10dc17__CheqSeq_500_bp_LC_R2.fastq.gz
Processing reads on 10 cores in single-end mode ...
Finished in 273.00 s (2 us/read; 24.27 M reads/minute).

=== Summary ===

Total reads processed:             110,407,162
Reads with adapters:               105,602,464 (95.6%)
Reads that were too short:           1,037,617 (0.9%)
Reads that were too long:            5,466,728 (5.0%)
Reads written (passing filters):   103,902,817 (94.1%)

Total basepairs processed: 5,409,950,938 bp
Total written (filtered):  1,766,347,889 bp (32.6%)

=== Adapter 2 ===

Sequence: AATTAATTCGGGCCCCGGTCC...GATCGGCGCGCCTGCTCG; Type: linked; Length: 21+18; 5' trimmed: 105602464 times; 3' trimmed: 105602464 times

No. of allowed errors:
0-9 bp: 0; 10-19 bp: 1; 20-21 bp: 2

No. of allowed errors:
0-9 bp: 0; 10-18 bp: 1

Overview of removed sequences at 5' e

In [8]:
# Q30 filtering
for file in 02.clean_reads/CSE__10dc17__CheqSeq_500_bp_LC_R*_BC.fastq.gz
do
fastp -e 30 --disable_length_filtering -h /dev/null/fastp.html -j /dev/null/fastp.json -w 8 \
      -o ${file%_*}_BCq30.fastq.gz \
      -i $file
done

Detecting adapter sequence for read1...
No adapter detected for read1

Read1 before filtering:
total reads: 107062265
total bases: 856498120
Q20 bases: 842534947(98.3697%)
Q30 bases: 810959428(94.6832%)

Read1 after filtering:
total reads: 104655282
total bases: 837242256
Q20 bases: 828559842(98.963%)
Q30 bases: 801375395(95.7161%)

Filtering result:
reads passed filter: 104655282
reads failed due to low quality: 2406983
reads failed due to too many N: 0
reads with adapter trimmed: 0
bases trimmed due to adapters: 0

Duplication rate (may be overestimated since this is SE data): 0%

JSON report: /dev/null/fastp.json
HTML report: /dev/null/fastp.html

fastp -e 30 --disable_length_filtering -h /dev/null/fastp.html -j /dev/null/fastp.json -w 8 -o 02.clean_reads/CSE__10dc17__CheqSeq_500_bp_LC_R1_BCq30.fastq.gz -i 02.clean_reads/CSE__10dc17__CheqSeq_500_bp_LC_R1_BC.fastq.gz 
fastp v0.20.0, time used: 185 seconds
Detecting adapter sequence for read1...
No adapter detected for read1

Read1 be

In [16]:
# output one line with total number of reads, number of unique reads, percentage of unique reads, most common sequence, number of occurrence of the most common sequence and frequency of the most common sequence.
for file in 02.clean_reads/*_BCq30.fastq.gz
do
zcat $file | awk '((NR-2)%4==0){read=$1;total++;count[read]++}END{for(read in count){if(!max||count[read]>max) {max=count[read];maxRead=read};if(count[read]==1){unique++}};print total,unique,unique*100/total,maxRead,count[maxRead],count[maxRead]*100/total}'
done

104655282 1217 0.00116287 AAGAATCG 3208727 3.066
101391868 1310014 1.29203 ACCTGACACTACAGGTC 19493 0.0192254


In [ ]:
cat Enh-freeBC.tsv | while read line
do
y=$(echo $line | awk '{print $2}')
zgrep $y 02.clean_reads/CSE__10dc17__CheqSeq_500_bp_LC_R1_BCq30.fastq.gz | wc -l
done
# ~94% of R1 are known BCs

## 03.enhancer_bc_assignment

In [21]:
SAMPLE_NAME="CSE__10dc17__CheqSeq_500_bp_LC"

### Cutadapt Strategy

In [22]:
zcat 02.clean_reads/${SAMPLE_NAME}_R1_BCq30.fastq.gz|head

@A00311:74:HMLK5DMXX:1:1101:1615:1000 1:N:0:CCAACAAT
TTCAGTGG
+
FFFFFFFF
@A00311:74:HMLK5DMXX:1:1101:1850:1000 1:N:0:CCAACAAT
TTACGTCC
+
FFFFFFFF
@A00311:74:HMLK5DMXX:1:1101:2193:1000 1:N:0:CCAACAAT
ATATCACG

gzip: stdout: Broken pipe


In [ ]:
# From the barcode reads,
# Create table with following content:
# Read ID<TAB>Enhancer Barcode
paste \
   <( zcat 02.clean_reads/${SAMPLE_NAME}_R1_BCq30.fastq.gz | sed -n '1~4p' | awk '{print $1}' | sed 's/^@//' ) \
   <( zcat 02.clean_reads/${SAMPLE_NAME}_R1_BCq30.fastq.gz | sed -n '2~4p' ) \
   | sort -k1 \
   | gzip -c \
   > 03.enhancer_bc_assignment/${SAMPLE_NAME}_readID_enhancerBarcode.tsv.gz

In [ ]:
zcat 03.enhancer_bc_assignment/${SAMPLE_NAME}_readID_enhancerBarcode.tsv.gz | head

In [ ]:
# From the barcode reads,
# Create table with following content:
# Read ID<TAB>Barcode
paste \
   <( zcat 02.clean_reads/${SAMPLE_NAME}_R2_BCq30.fastq.gz | sed -n '1~4p' | awk '{print $1}' | sed 's/^@//' ) \
   <( zcat 02.clean_reads/${SAMPLE_NAME}_R2_BCq30.fastq.gz | sed -n '2~4p' ) \
   | sort -k1 \
   | gzip -c \
   > 03.enhancer_bc_assignment/${SAMPLE_NAME}_readID_barcode.tsv.gz

In [ ]:
zcat 03.enhancer_bc_assignment/${SAMPLE_NAME}_readID_barcode.tsv.gz | head

In [ ]:
zcat 03.enhancer_bc_assignment/${SAMPLE_NAME}_readID_barcode.tsv.gz | wc -l

In [ ]:
zcat 03.enhancer_bc_assignment/${SAMPLE_NAME}_readID_barcode.tsv.gz | sort -k2 | cut -f2 | uniq | wc -l

In [ ]:
# Create table with following content:
# Read ID<TAB>Enhancer Barcode<TAB>Barcode
awk -F '\t' -v OFS='\t' 'FNR==NR{a[$1]=$2 FS $3;next}{ print $0, a[$1]}' \
   <( zcat 03.enhancer_bc_assignment/${SAMPLE_NAME}_readID_enhancerBarcode.tsv.gz ) \
   <( zcat 03.enhancer_bc_assignment/${SAMPLE_NAME}_readID_barcode.tsv.gz ) \
   | awk  '$3!=""' \
   | cut -f1-3 \
   | gzip -c \
   > 03.enhancer_bc_assignment/${SAMPLE_NAME}_readID_enhancerBarcode_barcode_coupled.tsv.gz

In [ ]:
zcat 03.enhancer_bc_assignment/${SAMPLE_NAME}_readID_enhancerBarcode_barcode_coupled.tsv.gz | head

In [ ]:
cat Enh-freeBC.tsv | wc -l

In [ ]:
# Replace enhancer barcode by the enhancer
# Create table with following content:
# Enhancer<TAB>Barcode
awk -F '\t' -v OFS='\t' 'FNR==NR{a[$2]=$1; next}{ print a[$2], $3}' \
   <( cat Enh-freeBC.tsv ) \
   <( zcat 03.enhancer_bc_assignment/${SAMPLE_NAME}_readID_enhancerBarcode_barcode_coupled.tsv.gz ) \
   | gzip -c \
   > 03.enhancer_bc_assignment/${SAMPLE_NAME}_enhancer_barcode_coupled.tsv.gz

In [20]:
# cat Enh-freeBC.tsv | awk '{print $2}'

## Remove empty couples

In [ ]:
zcat 03.enhancer_bc_assignment/${SAMPLE_NAME}_enhancer_barcode_coupled.tsv.gz \
   | awk -F'\t' '$1!=""' \
   | gzip -c \
   > 03.enhancer_bc_assignment/${SAMPLE_NAME}_enhancer_barcode_coupled_cleaned.tsv.gz

In [ ]:
zcat 03.enhancer_bc_assignment/${SAMPLE_NAME}_enhancer_barcode_coupled_cleaned.tsv.gz | wc -l

## Keep only unique couples

In [ ]:
zcat 03.enhancer_bc_assignment/${SAMPLE_NAME}_enhancer_barcode_coupled_cleaned.tsv.gz \
   | sort -k1 \
   | uniq \
   > 03.enhancer_bc_assignment/${SAMPLE_NAME}_enhancer_barcode_coupled_cleaned_unique.tsv.gz

In [ ]:
gzip 03.enhancer_bc_assignment/${SAMPLE_NAME}_enhancer_barcode_coupled_cleaned_unique.tsv

In [ ]:
zcat 03.enhancer_bc_assignment/${SAMPLE_NAME}_enhancer_barcode_coupled_cleaned_unique.tsv.gz | wc -l

In [ ]:
zcat 03.enhancer_bc_assignment/${SAMPLE_NAME}_enhancer_barcode_coupled_cleaned_unique.tsv.gz|head

## Remove barcodes linked to multiple enhancers

In [ ]:
zcat 03.enhancer_bc_assignment/${SAMPLE_NAME}_enhancer_barcode_coupled_cleaned_unique.tsv.gz \
   | awk -F'\t' '
   {
      if(a[$2]) {
         a[$2]=a[$2]":"$1; 
      } else {
         a[$2]=$1;
      }
   } END {
      for (i in a) {
         n=split(a[i],b,":")
         if(n == 1)
            print a[i], i;
      }
   }' OFS='\t' | gzip -c \
   > 03.enhancer_bc_assignment/${SAMPLE_NAME}_enhancer_barcode_coupled_cleaned_unique_1to1.tsv.gz

In [3]:
echo 03.enhancer_bc_assignment/${SAMPLE_NAME}_enhancer_barcode_coupled_cleaned_unique_1to1.tsv.gz

03.enhancer_bc_assignment/_enhancer_barcode_coupled_cleaned_unique_1to1.tsv.gz


In [4]:
zcat 03.enhancer_bc_assignment/${SAMPLE_NAME}_enhancer_barcode_coupled_cleaned_unique_1to1.tsv.gz | wc -l

gzip: 03.enhancer_bc_assignment/_enhancer_barcode_coupled_cleaned_unique_1to1.tsv.gz: No such file or directory
0


## 04.sequencing_saturation

In [3]:
# Extract DNA sequences from fastq
zcat 02.clean_reads/CSE__10dc17__CheqSeq_500_bp_LC_R2_BCq30.fastq.gz | sed -n '2~4p' > 04.sequencing_saturation/CSE__10dc17__CheqSeq_500_bp_LC_R2_BCq30.txt

In [4]:
# Subsample, count total and unique reads
start=`date +%s`

nreads=$(cat 04.sequencing_saturation/CSE__10dc17__CheqSeq_500_bp_LC_R2_BCq30.txt | wc -l)
 # Start by removing parallel instructions file
rm 03.sequencing_saturation/parallel.txt
 # Remove any previoulsy created sat values for that file
rm 04.sequencing_saturation/CSE__10dc17__CheqSeq_500_bp_LC_R2_sat.txt
    for i in $(seq 0 200000 $nreads)
    do
     # Print number of total and unique reads for every subsamples
    echo "echo $i \$(shuf -n $(($i)) 04.sequencing_saturation/CSE__10dc17__CheqSeq_500_bp_LC_R2_BCq30.txt | sort | uniq | wc -l) >> 04.sequencing_saturation/CSE__10dc17__CheqSeq_500_bp_LC_R2_sat.txt" \
    >> 04.sequencing_saturation/parallel.txt
    done
cat 04.sequencing_saturation/parallel.txt | parallel -j 16
 # Put values in numerical order
cat 04.sequencing_saturation/CSE__10dc17__CheqSeq_500_bp_LC_R2_sat.txt | sort -g > 04.sequencing_saturation/CSE__10dc17__CheqSeq_500_bp_LC_R2_satsort.txt && mv 04.sequencing_saturation/CSE__10dc17__CheqSeq_500_bp_LC_R2_satsort.txt 04.sequencing_saturation/CSE__10dc17__CheqSeq_500_bp_LC_R2_sat.txt

end=`date +%s`
echo $((end-start))

rm: cannot remove ‘03.sequencing_saturation/parallel.txt’: No such file or directory
rm: cannot remove ‘04.sequencing_saturation/CSE__10dc17__CheqSeq_500_bp_LC_R2_sat.txt’: No such file or directory
Academic tradition requires you to cite works you base your article on.
When using programs that use GNU Parallel to process data for publication
please cite:

  O. Tange (2011): GNU Parallel - The Command-Line Power Tool,
  ;login: The USENIX Magazine, February 2011:42-47.

This helps funding further development; AND IT WON'T COST YOU A CENT.
If you pay 10000 EUR you should feel free to use GNU Parallel without citing.

To silence the citation notice: run 'parallel --bibtex'.

10155
